In [1]:
import pandas as pd

In [2]:
import pickle as pkl

In [3]:
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesClassifier

In [4]:
from sklearn.ensemble import ExtraTreesRegressor

In [5]:
with open('features_juli_with_support.pkl', 'rb') as file:
    train = pkl.load(file)
    

In [6]:
with open('test_features_juli_with_support.pkl', 'rb') as file:
    test = pkl.load(file)

# Con mis features

In [7]:
train.columns

Index(['visited site', 'generic listing', 'conversion', 'viewed product',
       'person', 'tiempo_promedio_por_dia', 'label', 'modelo_mas_visto',
       'condition_mode', 'storage_mode', 'campaing_mode', 'cant_model_vistos',
       'event_support'],
      dtype='object')

In [8]:
train.shape

(19414, 13)

In [9]:
X_train = train[['visited site', 'generic listing', 'conversion', 'viewed product',
        'tiempo_promedio_por_dia','modelo_mas_visto',
       'condition_mode', 'storage_mode', 'campaing_mode', 'cant_model_vistos',
       'event_support']]

In [10]:
y_train = train.label

In [11]:
#split del dataset para entrenamiento y pruebas, 80% y 20% respectivamente
Xtrn, Xtest, Ytrn, Ytest = train_test_split(X_train,y_train,test_size=0.2)

In [12]:
model = ExtraTreesRegressor(n_estimators=10, max_features='auto', min_samples_leaf=1, min_samples_split=4, bootstrap=True)



In [13]:
model.fit(Xtrn,Ytrn)


AttributeError: 'ExtraTreesRegressor' object has no attribute 'predict_proba'

In [14]:
pred= model.predict(Xtest)

In [26]:
pred.shape

(3883, 2)

In [27]:
Ytest.shape

(3883,)

In [16]:
roc_auc_score(Ytest, pred)

0.6094624253566491

In [17]:
from sklearn.model_selection import GridSearchCV

In [18]:
help(ExtraTreesRegressor)

Help on class ExtraTreesRegressor in module sklearn.ensemble.forest:

class ExtraTreesRegressor(ForestRegressor)
 |  An extra-trees regressor.
 |  
 |  This class implements a meta estimator that fits a number of
 |  randomized decision trees (a.k.a. extra-trees) on various sub-samples
 |  of the dataset and uses averaging to improve the predictive accuracy
 |  and control over-fitting.
 |  
 |  Read more in the :ref:`User Guide <forest>`.
 |  
 |  Parameters
 |  ----------
 |  n_estimators : integer, optional (default=10)
 |      The number of trees in the forest.
 |  
 |      .. versionchanged:: 0.20
 |         The default value of ``n_estimators`` will change from 10 in
 |         version 0.20 to 100 in version 0.22.
 |  
 |  criterion : string, optional (default="mse")
 |      The function to measure the quality of a split. Supported criteria
 |      are "mse" for the mean squared error, which is equal to variance
 |      reduction as feature selection criterion, and "mae" for the 

In [19]:
param_grid = { 
          "n_estimators"      : [50,100,150],
          "max_features"      : ["auto"],
          "min_samples_split" : [2,3,4,5] ,
          "bootstrap": [True]
          }
grid = GridSearchCV(model, param_grid, n_jobs=-1, cv=3, verbose=1)

grid.fit(Xtrn, Ytrn)


Fitting 3 folds for each of 12 candidates, totalling 36 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed:  6.4min finished


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=ExtraTreesRegressor(bootstrap=True, criterion='mse', max_depth=None,
          max_features='auto', max_leaf_nodes=None,
          min_impurity_decrease=0.0, min_impurity_split=None,
          min_samples_leaf=1, min_samples_split=4,
          min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
          oob_score=False, random_state=None, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'n_estimators': [50, 100, 150], 'max_features': ['auto'], 'min_samples_split': [2, 3, 4, 5], 'bootstrap': [True]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [20]:
grid.best_score_

-0.019243671383476785

In [21]:
grid.best_params_

{'bootstrap': True,
 'max_features': 'auto',
 'min_samples_split': 5,
 'n_estimators': 150}

In [22]:
roc_auc_score(Ytest, grid.predict(Xtest))

0.7223531004603303

In [29]:
model = ExtraTreesRegressor(bootstrap = True,
 max_features = 'auto',
 min_samples_split= 5,
 n_estimators = 150)

In [30]:
model.fit(Xtrn, Ytrn)

ExtraTreesRegressor(bootstrap=True, criterion='mse', max_depth=None,
          max_features='auto', max_leaf_nodes=None,
          min_impurity_decrease=0.0, min_impurity_split=None,
          min_samples_leaf=1, min_samples_split=5,
          min_weight_fraction_leaf=0.0, n_estimators=150, n_jobs=None,
          oob_score=False, random_state=None, verbose=0, warm_start=False)

In [32]:
roc_auc_score(Ytest, model.predict(Xtest))

0.7237205523964256

In [35]:
Xtrn.columns

Index(['visited site', 'generic listing', 'conversion', 'viewed product',
       'tiempo_promedio_por_dia', 'modelo_mas_visto', 'condition_mode',
       'storage_mode', 'campaing_mode', 'cant_model_vistos', 'event_support'],
      dtype='object')

In [33]:
model.feature_importances_

array([0.10913236, 0.11218708, 0.0335432 , 0.1306351 , 0.15259867,
       0.03934962, 0.06388354, 0.10192728, 0.        , 0.06891556,
       0.18782759])

In [36]:
model.fit(Xtrn[['visited site', 'generic listing', 'viewed product',
       'tiempo_promedio_por_dia', 'storage_mode', 'event_support']], Ytrn)

ExtraTreesRegressor(bootstrap=True, criterion='mse', max_depth=None,
          max_features='auto', max_leaf_nodes=None,
          min_impurity_decrease=0.0, min_impurity_split=None,
          min_samples_leaf=1, min_samples_split=5,
          min_weight_fraction_leaf=0.0, n_estimators=150, n_jobs=None,
          oob_score=False, random_state=None, verbose=0, warm_start=False)

In [38]:
roc_auc_score(Ytest,model.predict(Xtest[['visited site', 'generic listing', 'viewed product',
       'tiempo_promedio_por_dia', 'storage_mode', 'event_support']]))

0.7434121453104736

In [39]:
param_grid = { 
          "n_estimators"      : [150],
          "max_features"      : ["auto"],
          "min_samples_split" : [2,3,4,5] ,
          "bootstrap": [True]
          }
grid = GridSearchCV(model, param_grid, n_jobs=-1, cv=3, verbose=1)

grid.fit(Xtrn[['visited site', 'generic listing', 'viewed product',
       'tiempo_promedio_por_dia', 'storage_mode', 'event_support']], Ytrn)

Fitting 3 folds for each of 4 candidates, totalling 12 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:  3.5min finished


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=ExtraTreesRegressor(bootstrap=True, criterion='mse', max_depth=None,
          max_features='auto', max_leaf_nodes=None,
          min_impurity_decrease=0.0, min_impurity_split=None,
          min_samples_leaf=1, min_samples_split=5,
          min_weight_fraction_leaf=0.0, n_estimators=150, n_jobs=None,
          oob_score=False, random_state=None, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'n_estimators': [150], 'max_features': ['auto'], 'min_samples_split': [2, 3, 4, 5], 'bootstrap': [True]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [41]:
roc_auc_score(Ytest, grid.predict(Xtest[['visited site', 'generic listing', 'viewed product',
       'tiempo_promedio_por_dia', 'storage_mode', 'event_support']]))

0.7316608947225904

In [55]:
test.columns

Index(['visited site', 'generic listing', 'conversion', 'viewed product',
       'person', 'tiempo_promedio_por_dia', 'modelo_mas_visto',
       'condition_mode', 'storage_mode', 'campaing_mode', 'cant_model_vistos',
       'event_support'],
      dtype='object')

In [23]:
x_test = test[['visited site', 'generic listing', 'conversion', 'viewed product',
        'tiempo_promedio_por_dia', 'modelo_mas_visto',
       'condition_mode', 'storage_mode', 'campaing_mode', 'cant_model_vistos',
       'event_support']]

In [24]:
sum(pd.isnull(x_test['event_support']))

0

In [25]:
x_test['viewed product'].fillna(x_test['viewed product'].mean(), inplace = True)

/home/july/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:3660: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [26]:
x_test['visited site'].fillna(x_test['visited site'].mean(), inplace = True)

/home/july/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:3660: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [27]:
x_test['generic listing'].fillna(x_test['generic listing'].mean(), inplace = True)

/home/july/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:3660: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [28]:
x_test['conversion'].fillna(x_test['conversion'].mean(), inplace = True)

/home/july/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:3660: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [79]:
pred_porta = grid.predict_proba(x_test)[:,1]

In [77]:
to_puvlic = pd.DataFrame()

In [78]:
to_puvlic['person'] = test.person

In [80]:
to_puvlic['label'] = pred_porta

In [82]:
to_puvlic.to_csv('extraTreesJuli.csv', index = False)

# Con features de las chicas

In [42]:
with open('features_chicas.pkl','rb') as file:
    features = pkl.load(file)

In [85]:
features.columns

Index(['person', 'cant_eventos_totales', 'cant_ach', 'cant_bl', 'cant_ch',
       'cant_cv', 'cant_gl', 'cant_l', 'cant_seh', 'cant_s_p', 'cant_sp',
       'cant_vp', 'cant_vs', 'Returning', 'tiempo_total',
       'tiempo_new_returning', 'channel_frecuente', 'device_frecuente', 'q1',
       'q2', 'q3', 'q4', 'q5', 'q6', 'q7', 'q8', 'q9', 'q10'],
      dtype='object')

In [43]:
features['device_frecuente'].fillna(features['device_frecuente'].mode()[0], inplace=True)

In [44]:
features[ 'Returning'].fillna(features['Returning'].mode()[0], inplace=True)

In [45]:
import category_encoders as ce

In [46]:
te_channel_frecuente = ce.TargetEncoder(handle_unknown='ignore')

In [47]:
label_train = pd.read_csv('data/fiuba-trocafone-tp2-final-set/labels_training_set.csv')

In [48]:
train_features = pd.merge(features, label_train, on='person', how='right')

In [49]:
train_features.columns

Index(['person', 'cant_eventos_totales', 'cant_ach', 'cant_bl', 'cant_ch',
       'cant_cv', 'cant_gl', 'cant_l', 'cant_seh', 'cant_s_p', 'cant_sp',
       'cant_vp', 'cant_vs', 'Returning', 'tiempo_total',
       'tiempo_new_returning', 'channel_frecuente', 'device_frecuente', 'q1',
       'q2', 'q3', 'q4', 'q5', 'q6', 'q7', 'q8', 'q9', 'q10', 'label'],
      dtype='object')

In [50]:
sum(pd.isnull(train_features['device_frecuente']))

0

In [51]:
train_features['Returning'].fillna(train_features['Returning'].mode()[0], inplace=True)

In [52]:
train_features['device_frecuente'].fillna(train_features['device_frecuente'].mode()[0], inplace=True)

In [53]:
x_train_features = train_features[[ 'cant_eventos_totales', 'cant_ach', 'cant_bl', 'cant_ch',
       'cant_cv', 'cant_gl', 'cant_l', 'cant_seh', 'cant_s_p', 'cant_sp',
       'cant_vp', 'cant_vs', 'Returning', 'tiempo_total',
       'tiempo_new_returning', 'channel_frecuente', 'device_frecuente', 'q1',
       'q2', 'q3', 'q4', 'q5', 'q6', 'q7', 'q8', 'q9', 'q10']]

In [54]:
y_train_features = train_features.label

In [55]:
x_train_features.channel_frecuente = te_channel_frecuente.fit_transform(pd.DataFrame(x_train_features.channel_frecuente), y_train_features)

/home/july/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:3110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [56]:
te_device_frecuente = ce.TargetEncoder(handle_unknown='ignore')

In [57]:
x_train_features.device_frecuente = te_device_frecuente.fit_transform(pd.DataFrame(x_train_features.device_frecuente), y_train_features)

/home/july/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:3110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [58]:
x_train_features.shape

(19414, 27)

In [59]:
#split del dataset para entrenamiento y pruebas, 80% y 20% respectivamente
Xtrn2, Xtest2, Ytrn2, Ytest2 = train_test_split(x_train_features,y_train_features,test_size=0.2)

In [63]:
param_grid2C = { 
          "n_estimators"      : [1,2,3,4,5,6,7,8,9,10],
          "max_features"      : ["auto"],
          #"min_samples_split" : [2,3,4,5] ,
          "bootstrap": [True]
          }
grid2C = GridSearchCV(model, param_grid2C, n_jobs=-1, cv=3, verbose=1)

grid2C.fit(Xtrn2, Ytrn2)


Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:  1.3min finished


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=ExtraTreesRegressor(bootstrap=True, criterion='mse', max_depth=None,
          max_features='auto', max_leaf_nodes=None,
          min_impurity_decrease=0.0, min_impurity_split=None,
          min_samples_leaf=1, min_samples_split=5,
          min_weight_fraction_leaf=0.0, n_estimators=150, n_jobs=None,
          oob_score=False, random_state=None, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'n_estimators': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 'max_features': ['auto'], 'bootstrap': [True]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [64]:
grid2C.best_score_

-0.016188412588598634

In [65]:
roc_auc_score(Ytest2, grid2C.predict(Xtest2))

0.7494028309337222

In [68]:
grid2C.best_params_

{'bootstrap': True, 'max_features': 'auto', 'n_estimators': 10}

In [69]:
param_grid3C = { 
          "n_estimators"      : [10,20,30,40,50,60,70,80,90,100],
          "max_features"      : ["auto"],
          #"min_samples_split" : [2,3,4,5] ,
          "bootstrap": [True]
          }
grid3C = GridSearchCV(model, param_grid3C, n_jobs=-1, cv=3, verbose=1)

grid3C.fit(Xtrn2, Ytrn2)


Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:  7.7min finished


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=ExtraTreesRegressor(bootstrap=True, criterion='mse', max_depth=None,
          max_features='auto', max_leaf_nodes=None,
          min_impurity_decrease=0.0, min_impurity_split=None,
          min_samples_leaf=1, min_samples_split=5,
          min_weight_fraction_leaf=0.0, n_estimators=150, n_jobs=None,
          oob_score=False, random_state=None, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'n_estimators': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100], 'max_features': ['auto'], 'bootstrap': [True]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [70]:
grid3C.best_score_

0.04649705472364788

In [72]:
roc_auc_score(Ytest2,grid3C.predict(Xtest2))

0.8059753356418201

In [73]:
grid3C.best_params_

{'bootstrap': True, 'max_features': 'auto', 'n_estimators': 80}

In [77]:
help(ExtraTreesRegressor)

Help on class ExtraTreesRegressor in module sklearn.ensemble.forest:

class ExtraTreesRegressor(ForestRegressor)
 |  An extra-trees regressor.
 |  
 |  This class implements a meta estimator that fits a number of
 |  randomized decision trees (a.k.a. extra-trees) on various sub-samples
 |  of the dataset and uses averaging to improve the predictive accuracy
 |  and control over-fitting.
 |  
 |  Read more in the :ref:`User Guide <forest>`.
 |  
 |  Parameters
 |  ----------
 |  n_estimators : integer, optional (default=10)
 |      The number of trees in the forest.
 |  
 |      .. versionchanged:: 0.20
 |         The default value of ``n_estimators`` will change from 10 in
 |         version 0.20 to 100 in version 0.22.
 |  
 |  criterion : string, optional (default="mse")
 |      The function to measure the quality of a split. Supported criteria
 |      are "mse" for the mean squared error, which is equal to variance
 |      reduction as feature selection criterion, and "mae" for the 

In [78]:
param_grid4C = { 
          "n_estimators"      : [80],
          "max_features"      : ["auto","sqrt", "log2"],
          "min_samples_split" : [2,3,4,5] ,
          "bootstrap": [True,False]
          }
grid4C = GridSearchCV(model, param_grid4C, n_jobs=-1, cv=2, verbose=1)

grid4C.fit(Xtrn2, Ytrn2)

Fitting 2 folds for each of 24 candidates, totalling 48 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  48 out of  48 | elapsed:  9.9min finished


GridSearchCV(cv=2, error_score='raise-deprecating',
       estimator=ExtraTreesRegressor(bootstrap=True, criterion='mse', max_depth=None,
          max_features='auto', max_leaf_nodes=None,
          min_impurity_decrease=0.0, min_impurity_split=None,
          min_samples_leaf=1, min_samples_split=5,
          min_weight_fraction_leaf=0.0, n_estimators=150, n_jobs=None,
          oob_score=False, random_state=None, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'n_estimators': [80], 'max_features': ['auto', 'sqrt', 'log2'], 'min_samples_split': [2, 3, 4, 5], 'bootstrap': [True, False]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [79]:
grid4C.best_score_

0.05415214070383478

In [80]:
grid4C.best_params_

{'bootstrap': True,
 'max_features': 'sqrt',
 'min_samples_split': 5,
 'n_estimators': 80}

In [113]:
roc_auc_score(Ytest2, grid4C.predict(Xtest2))

0.8020065609622744

In [83]:
model = ExtraTreesRegressor(bootstrap = True,
 max_features = 'sqrt',
 min_samples_split = 5,
 n_estimators = 80)

In [84]:
model.fit(Xtrn2,Ytrn2)

ExtraTreesRegressor(bootstrap=True, criterion='mse', max_depth=None,
          max_features='sqrt', max_leaf_nodes=None,
          min_impurity_decrease=0.0, min_impurity_split=None,
          min_samples_leaf=1, min_samples_split=5,
          min_weight_fraction_leaf=0.0, n_estimators=80, n_jobs=None,
          oob_score=False, random_state=None, verbose=0, warm_start=False)

In [86]:
Xtrn2.columns

Index(['cant_eventos_totales', 'cant_ach', 'cant_bl', 'cant_ch', 'cant_cv',
       'cant_gl', 'cant_l', 'cant_seh', 'cant_s_p', 'cant_sp', 'cant_vp',
       'cant_vs', 'Returning', 'tiempo_total', 'tiempo_new_returning',
       'channel_frecuente', 'device_frecuente', 'q1', 'q2', 'q3', 'q4', 'q5',
       'q6', 'q7', 'q8', 'q9', 'q10'],
      dtype='object')

In [85]:
model.feature_importances_

array([0.0595408 , 0.05340405, 0.06479719, 0.1130123 , 0.03981237,
       0.05839894, 0.01048867, 0.05287804, 0.05645221, 0.02805691,
       0.06232582, 0.04246352, 0.03952069, 0.07084836, 0.04668135,
       0.04407682, 0.02826126, 0.0087554 , 0.00939686, 0.00815536,
       0.00799735, 0.01164598, 0.01372294, 0.01540209, 0.01479145,
       0.01824879, 0.02086446])

In [87]:
param_grid5C = { 
          "n_estimators"      : [80],
          "max_features"      : ["auto","sqrt", "log2"],
          "min_samples_split" : [2,3,4,5] ,
          "bootstrap": [True,False]
          }
grid5C = GridSearchCV(model, param_grid5C, n_jobs=-1, cv=2, verbose=1)

grid5C.fit(Xtrn2[['cant_bl', 'cant_ch', 'cant_vp',
       'tiempo_total']], Ytrn2)

Fitting 2 folds for each of 24 candidates, totalling 48 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  48 out of  48 | elapsed:  4.5min finished


GridSearchCV(cv=2, error_score='raise-deprecating',
       estimator=ExtraTreesRegressor(bootstrap=True, criterion='mse', max_depth=None,
          max_features='sqrt', max_leaf_nodes=None,
          min_impurity_decrease=0.0, min_impurity_split=None,
          min_samples_leaf=1, min_samples_split=5,
          min_weight_fraction_leaf=0.0, n_estimators=80, n_jobs=None,
          oob_score=False, random_state=None, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'n_estimators': [80], 'max_features': ['auto', 'sqrt', 'log2'], 'min_samples_split': [2, 3, 4, 5], 'bootstrap': [True, False]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [88]:
grid5C.best_score_

0.022339504604082315

In [90]:
roc_auc_score(Ytest2, grid5C.predict(Xtest2[['cant_bl', 'cant_ch', 'cant_vp',
       'tiempo_total']]))

0.7703930502399612

In [114]:
test_person = pd.read_csv('../7506_ml/data/trocafone_kaggle_test.csv')

In [115]:
test_person.shape

(19415, 1)

In [116]:
test2 = pd.merge(features , test_person, on='person', how='right')

In [117]:
test2.channel_frecuente = te_channel_frecuente.transform(pd.DataFrame(test2.channel_frecuente))

In [118]:
test2.device_frecuente = te_device_frecuente.transform(pd.DataFrame(test2.device_frecuente))

In [119]:
test2.columns

Index(['person', 'cant_eventos_totales', 'cant_ach', 'cant_bl', 'cant_ch',
       'cant_cv', 'cant_gl', 'cant_l', 'cant_seh', 'cant_s_p', 'cant_sp',
       'cant_vp', 'cant_vs', 'Returning', 'tiempo_total',
       'tiempo_new_returning', 'channel_frecuente', 'device_frecuente', 'q1',
       'q2', 'q3', 'q4', 'q5', 'q6', 'q7', 'q8', 'q9', 'q10'],
      dtype='object')

In [120]:
xtest2 = test2[['cant_eventos_totales', 'cant_ach', 'cant_bl', 'cant_ch',
       'cant_cv', 'cant_gl', 'cant_l', 'cant_seh', 'cant_s_p', 'cant_sp',
       'cant_vp', 'cant_vs', 'Returning', 'tiempo_total',
       'tiempo_new_returning', 'channel_frecuente', 'device_frecuente', 'q1',
       'q2', 'q3', 'q4', 'q5', 'q6', 'q7', 'q8', 'q9', 'q10']]

In [121]:
sum(pd.isnull(xtest2['device_frecuente']))

0

In [122]:
xtest2['channel_frecuente'].fillna(xtest2['channel_frecuente'].mode()[0], inplace=True)

/home/july/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:3660: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [123]:
pred = grid4C.predict(xtest2)

In [124]:
to_public = pd.DataFrame()

In [125]:
to_public['person'] = test2.person

In [126]:
to_public['label'] = pred

In [127]:
to_public.to_csv('extraTreesRM.csv', index=False)

In [125]:
grid2C.best_params_

{'bootstrap': True,
 'max_features': 'auto',
 'min_samples_split': 5,
 'n_estimators': 100}

# Con el de Mati

In [91]:
df = pkl.load(open('Forma1/feature.pkl','rb'))

In [92]:
df.columns

Index(['person', 'label', 'storage_mode', 'event_ad campaign hit',
       'event_brand listing', 'event_checkout', 'event_conversion',
       'event_generic listing', 'event_lead', 'event_search engine hit',
       'event_searched products', 'event_staticpage', 'event_viewed product',
       'event_visited site', 'new_vs_returning_New',
       'new_vs_returning_Returning', 'condition_Bom',
       'condition_Bom - Sem Touch ID', 'condition_Excelente',
       'condition_Muito Bom', 'condition_Novo', 'time_cos_by_day'],
      dtype='object')

In [93]:
xTrain = df[['storage_mode', 'event_ad campaign hit',
       'event_brand listing', 'event_checkout', 'event_conversion',
       'event_generic listing', 'event_lead', 'event_search engine hit',
       'event_searched products', 'event_staticpage', 'event_viewed product',
       'event_visited site', 'new_vs_returning_New',
       'new_vs_returning_Returning', 'condition_Bom',
       'condition_Bom - Sem Touch ID', 'condition_Excelente',
       'condition_Muito Bom', 'condition_Novo', 'time_cos_by_day']]

In [94]:
sum(pd.isnull(xTrain['time_cos_by_day']))

0

In [95]:
xTrain['condition_Novo'].fillna(xTrain['condition_Novo'].mode()[0],inplace=True)

/home/july/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:3660: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [96]:
xTrain['condition_Muito Bom'].fillna(xTrain['condition_Muito Bom'].mode()[0],inplace=True)

/home/july/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:3660: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [97]:
xTrain['condition_Excelente'].fillna(xTrain['condition_Excelente'].mode()[0],inplace=True)

/home/july/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:3660: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [98]:
xTrain['new_vs_returning_New'].fillna(xTrain['new_vs_returning_New'].mode()[0],inplace=True)

/home/july/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:3660: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [99]:
xTrain['new_vs_returning_Returning'].fillna(xTrain['new_vs_returning_Returning'].mode()[0],inplace=True)

/home/july/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:3660: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [100]:
xTrain['condition_Bom - Sem Touch ID'].fillna(xTrain['condition_Bom - Sem Touch ID'].mode()[0],inplace=True)

/home/july/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:3660: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [101]:
xTrain['condition_Bom'].fillna(xTrain['condition_Bom'].mode()[0],inplace=True)

/home/july/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:3660: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [102]:
yTrain = df.label

In [103]:
#split del dataset para entrenamiento y pruebas, 80% y 20% respectivamente
Xtrn3, Xtest3, Ytrn3, Ytest3 = train_test_split(xTrain,yTrain,test_size=0.2)

In [104]:
model = ExtraTreesRegressor(n_estimators=10, max_features='auto', min_samples_leaf=1, min_samples_split=4, bootstrap=True)

In [105]:
model.fit(Xtrn3,Ytrn3)

ExtraTreesRegressor(bootstrap=True, criterion='mse', max_depth=None,
          max_features='auto', max_leaf_nodes=None,
          min_impurity_decrease=0.0, min_impurity_split=None,
          min_samples_leaf=1, min_samples_split=4,
          min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
          oob_score=False, random_state=None, verbose=0, warm_start=False)

In [106]:
roc_auc_score(Ytest3, model.predict(Xtest3))

0.7052789362090512

In [107]:
param_gridM = { 
          "n_estimators"      : [50,100,150],
          "max_features"      : ["auto"],
          "min_samples_split" : [2,3,4,5] ,
          "bootstrap": [True]
          }
gridM = GridSearchCV(model, param_gridM, n_jobs=-1, cv=3, verbose=10)

gridM.fit(Xtrn3, Ytrn3)


Fitting 3 folds for each of 12 candidates, totalling 36 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   47.1s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  3.4min
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:  4.5min
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed:  7.1min
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed: 10.3min
[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed: 13.3min finished


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=ExtraTreesRegressor(bootstrap=True, criterion='mse', max_depth=None,
          max_features='auto', max_leaf_nodes=None,
          min_impurity_decrease=0.0, min_impurity_split=None,
          min_samples_leaf=1, min_samples_split=4,
          min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
          oob_score=False, random_state=None, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'n_estimators': [50, 100, 150], 'max_features': ['auto'], 'min_samples_split': [2, 3, 4, 5], 'bootstrap': [True]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=10)

In [108]:
gridM.best_score_

0.03796428853722926

In [110]:
gridM.best_params_

{'bootstrap': True,
 'max_features': 'auto',
 'min_samples_split': 5,
 'n_estimators': 150}

In [112]:
roc_auc_score(Ytest3, gridM.predict(Xtest3))

0.7859001362034346